# Dynamic programming

### Memoization: The Fibonacci problem
Take the problem of computing the nth term of the Fibonacci sequence. A slow solution is

In [7]:
#Slow
def fib(n):
    if n <= 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

print(fib(7))
print(fib(20))

13
6765


The previous algorithm is slow because we repeat many computations, as it can be seen as a tree that divides in two at each branch until it gets to the base cases (which are also repeated many times). As each call to the function spawns two more calls, we have a number of calls: $1+1*2+2*2+4*2+\dots=2^0+2^1+2^2+2^3+\dots$, which in the worst case scenario has of the order of $n$ terms. Each call takes a unit of time and also a unit of space (memory), as it has to save each call on the stack for later use, but as there is a single processor, no more than $n$ calls will be in queue at any given time. Therefore, the time complexity of the algorithm is $O(2^n)$ and the space complexity is $O(n)$. 
<img src="fibo.png" width=400/>

However, we know that once we have, say, the result for fib(3)=2, we could use it whenever we need fib(3) instead of make a new call (and the tree of calls thereof) to compute it again. *Dynamic programming* is about dividing a problem into similar but smaller subproblems (like recursion), but then identifying when we have duplicate structures.

*Memoization* is the practice of storing information that, can save us from making duplicate computations, into some data structure for later use.

In [15]:
#Using a dictionary to store the results of calls
def fib_m(n, memo = {}):
    if n not in memo:
        if n <= 2:
            memo[n] = 1
        else:
            memo[n] = fib_m(n-1, memo) + fib_m(n-2, memo)
    #print(memo)
    return memo[n]

print(fib_m(7))
print(fib_m(20))
print(fib_m(50))

13
6765
12586269025


We see now that we only make a single call to the function with a particular argument and store the value in the dictionary for later use. This means that every time we make a call for an unsaved value $i$, this will spawn a single recursive call and a read from memory. This means our time and space complexities are now both $O(n)$.
<img src="memofib.png" width=400/>

### Memoization: The 2D traveler problem
Suppose we have a $m\times n$ 2D square grid and a traveler, that can only move down or right, has to go from the top-left corner to the bottom-right one. In how many ways is it possible to go from the initial to the final point?<br>
We begin by finding out how to state the problem in terms of smaller, but similar, problems. Note that as the traveler can only move down or right, after each move the size of the accessible space will decrease, and the problem shrinks:
<img src="trav1.png" width=300/>
<img src="trav2.png" width=300/>
<img src="trav3.png" width=300/>
As a base case we would have the problem of having either a $2\times 1$ or a $1\times 2$ grid, where there is single path. This problem can also be seen as a tree, where each node has the size of the accessible grid from that point, and its children are the two grid sizes that it can shrink to (or a single one if it gets to an edge).
<img src="travtree.png" width=300>

In [34]:
#Slow
def gridTraveler(m,n):
    if m == 0 or n == 0:
        return 0
    elif m == 1 and n == 1:
        return 1
    else:
        return gridTraveler(m-1,n) + gridTraveler(m,n-1)

print(gridTraveler(2,3))
print(gridTraveler(3,2))
print(gridTraveler(3,3))
#print(gridTraveler(18,18)) #Exceeds recursion depth

3
3
6


In the worst case scenario we see that this binary tree has a height of max$(m,n)$+1, so its time complexity is $O(2^{\text{max}(m,n)+1})$, while its space complexity is $O(\text{max}(m,n)+1)$. As in the Fibonacci case, we observe that there is duplicate work, as there are several ways to obtain the same reduced grid., additionally, we can use the vertical-horizontal symmetry, as the number of ways to traverse an $m\times n$ grid is the same number of ways to traverse an $n\times m$ grid. We will now memoize the problem by storing the values of the smaller problems ant taking the symmetry into account:

In [37]:
def gridTraveler_m(m,n, memo = {}):
    key = "%d,%d" % (max(m,n), min(m,n))
    if key not in memo:
        if m == 0 or n == 0:
            memo[key] = 0
        elif m == 1 and n == 1:
            memo[key] = 1
        else:
            memo[key] = gridTraveler_m(m-1,n, memo) + gridTraveler_m(m,n-1, memo) 
    #print(key)
    return memo[key]
        
print(gridTraveler_m(2,3))
print(gridTraveler_m(3,2))
print(gridTraveler_m(3,3))
print(gridTraveler_m(4,3))
print(gridTraveler_m(18,18))

3
3
6
10
2333606220


Now we continue to have a space complexity of order $O(\text{max}(m,n))$, but our time complexity has reduced to $O((\text{max}(m,n)+1)*\text{min}(m,n))$.

### Memoization: The canSum problem
The problem consists in the following: given a set of $n$ non-negative  integers $\{x_1,\dots,x_n\}$, we need to figure out if a target non-negative integer $m$ can be obtained by adding any combination of the numbers in the set (including repeats).<br>
For example, if the set is $\{5,3,4,7\}$ and the target is $7$, then the answer is True, because we can obtain $7$ either by taking just the $7$ in the set or by adding $3+4$. To obtain a recursive solution we note that we can substract all possible combinations of the set from the target, and if at any point we obtain $0$, then the result is True. Otherwise it is False. Note that if at any point the result is smaller than any of the numbers in the set we stop the substraction process, as it is no longer possible to substract any set member without obtaining a negative value. 
<img src="canSum1.png" width=500>

In [9]:
#Slow
def canSum(target, nums):
    if target == 0:
        return True
    elif target < min(nums):
        return False
    else:
        res = False
        for i in nums:
            res = res or canSum(target-i, nums)
        return res
    
print(canSum(7, [2,3]))
print(canSum(7, [5,3,4,7]))
print(canSum(7, [2,4]))
print(canSum(8, [2,3,5]))
#print(canSum(300, [7,14])) #Takes a long time

True
True
False
True


In the worst case (when 1 is in the set) the tree will have a height of $m$, and at each node it will branch $n$ times, that is, the time complexity is $O(n^m)$, while we might have at any given moment $m$ calls to the function (one for for each level of the tree), that is, the space complexity is $O(m)$. 

By looking at a tree, we can identify that there are duplicated structures in it, which means we can memoize it to avoid some recursive calls.
<img src="canSum2.png" width=600>

In [32]:
def canSum_m(target, nums, memo = {}):
    #print(memo)
    if target not in memo:
        #print(target)
        if target == 0:
            memo[target] = True
        elif target < min(nums):
            memo[target] = False
        else:
            res = False
            for i in nums:
                res = res or canSum_m(target-i, nums, memo)
            memo[target] = res
    return memo[target]

#NOTE: for some reason memo is sometimes retained from one call to the other!!!!!!!!
print(canSum_m(7, [2,3], {}))
print(canSum_m(7, [5,3,4,7], {}))
print(canSum_m(7, [2,4], {}))
print(canSum_m(8, [2,3,5], {}))
print(canSum_m(300, [7,14], {}))

True
True
False
True
False


Now that the algorithm is memoized we still have a space complexity of $O(m)$, but we need to compute that canSum for a particular target only once, and the number of targets is, at most, $m$, while for each of these call we need to read at most $n$ values from memory. That is, the time complexity becomes $O(m*n)$.

### Memoization recepie
First part
- Visualize the problem as a tree
- Implement a recursive algorithm
- Test it

Second part
- Add a memo object
- Store return values into the memo
- Use the stored values whenever they are available, instead of making a recursive call

### Tabulation: The Fibonacci problem
We solve again the Fibonacci problem, but this time we will start at 0:

In [1]:
#Slow
def fib(n):
    if n == 0 or n == 1:
        return n
    else:
        return fib(n-1) + fib(n-2)

print(fib(7))
print(fib(20))

13
6765


Again, the space complexity of the algorithm is $O(n)$ and the time complexity is $O(2^n)$. This time we will save again pervious results in memory to improve our time complexity, but weu will save them in a table (an 1D array in this case) and compute the values iteratively:

In [3]:
def fib_t(n):
    table = [0 for i in range(n+1)]
    table[1] = 1
    for i in range(2,n+1):
        table[i] = table[i-1] + table[i-2]
    return table[n]

print(fib_t(7))
print(fib_t(20))
print(fib_t(50))

13
6765
12586269025


Now, our space complexity continues to be $O(n)$, but our time complexity is also $O(n)$.

### Tabulation: The 2D traveler problem
In the original recursive method we had space complexity $O(\text{max}(m,n)+1)$ and time complexity $O(2^{\text{max}(m,n)+1})$. In this case we are goint to use an $m+1\times n+1$ table to store our results. We begin by initializing the base cases $0\times x$, $x\times0$, and $1\times1$, and proceed to fill the table iteratively by using the fact, which is clear in the recursive solution, that the number of ways to traverse a grid of size $i\times j$ is the sum of the number of ways to traverse a $i-1\times j$ grid, plus the number of ways to traverse a $i\times j-1$ grid: 

In [5]:
def gridTraveler_t(m,n):
    table = [[0 for j in range(n+1)] for i in range(m+1)]
    table[1][1] = 1
    for i in range(1,m+1):
        for j in range(1,n+1):
            if i*j > 1:
                table[i][j] = table[i-1][j] + table[i][j-1]
    return table[m][n]

print(gridTraveler_t(2,3))
print(gridTraveler_t(3,2))
print(gridTraveler_t(3,3))
print(gridTraveler_t(4,3))
print(gridTraveler_t(18,18))

3
3
6
10
2333606220


We have now reduced out time complexity to $O(m*n)$, while we have made a trade-off by increasing our space complexity to $O((m+1)*(n+1))$. In this case the trade-off is still worth it, as long as we have enough memory.

### Tabulation: The canSum problem
The original recursive algorithm had time complexity of $O(n^m)$ and space complexity of $O(m)$. We saw that whether a particular node gets a True or False value depends on the *or* reduction of the values of its children, with the True base case occurring for node $0$ and the False base case for any node value smaller than the smallest value in the set. To fill the rest of the table, remember that the sum a number with True lable and one of the lelements of the set is also true, so do a double iteration adding all the pairs of True numbers and set elements and set the position with this result as True too.  

In [50]:
def canSum_t(target, nums):
    table = [False for i in range(target+1)]
    table[0] = True
    for n in nums:
        table[n] = True
    for i in range(0, target):
        if table[i]:
            for j in nums:
                if i+j <= target:
                    #print(i,j)
                    table[i+j] = True
    return table[target]

print(canSum_t(7, [2,3]))
print(canSum_t(7, [5,3,4,7]))
print(canSum_t(7, [2,4]))
print(canSum_t(8, [2,3,5]))
print(canSum_t(300, [7,14]))

True
True
False
True
False


This version of the algorithm has space complexity of $O(m)$ and time complexity of $O(m*n)$.

### Tabulation recepie
- Visualize the problem as a table
- Size the table based on the inputs
- Initialize the known cases (usually this includes the base cases of the recursion)
- Use the recursion relation to fill the rest of the table iteratively